In [ ]:
%matplotlib inline
import pandas as pd
from pandas import DataFrame
from sodapy import Socrata
import matplotlib.pyplot as plt
from numpy import nan as NA

In [ ]:
plt.style.use('ggplot')

We'll use our `client` to connect to the DataSF API. For requests to the API for 311 data alone, I've found that a Socrata app token is unnecessary (I was not throttled).

In [ ]:
client = Socrata("data.sfgov.org", None)

In [ ]:
# Resource ID for 311 data
socrata_resource_id = "ktji-gk7t"

start_date = '2017-01-01T00:00:00'
end_date = '2018-01-01T00:00:00'
record_limit=1000000

In [ ]:
results = client.get(socrata_resource_id, 
                     where=f"requested_datetime between '{start_date}' and '{end_date}'", 
                     limit=record_limit)

In [ ]:
df = DataFrame.from_records(results)

## Initial Exploration & Data Cleaning

In [ ]:
df.shape

Personally, with DataFrames of many columns, I like to keep a list of columns for review, popping columns off as I validate them.

In [ ]:
columns_to_review = list(df.columns)
columns_to_review

In [ ]:
df.head()

### Columns to drop

`point` looks like GeoJSON data, and doesn't convey any more information for our analysis than `lat` and `long`. Working with GeoJSON data here will be difficult, so we drop it.

For this analysis, we don't care about `media_url` (the URL of any image, for instance, attached to the case).

In [ ]:
_ = df.drop(columns=['point', 'media_url'], inplace=True)

In [ ]:
columns_to_review.remove('point')
columns_to_review.remove('media_url')

In [ ]:
df[df.duplicated()].shape[0]

There are no duplicate records in the 311 data, at least when comparing the exact values of fields.

In [ ]:
df.isnull().sum()

We've got a few fields for which we have missing data:

* I'd expect missing `closed_date` to indicate that the case is still open. We should exclude some of these from analysis, but this is expected.
* Other fields, e.g. `lat`, `long`, and `police_district`, are all related. If we're missing a (lat, long), this means these other fields likely aren't populated. Let's confirm that the records for which these values are missing are _all_ missing.

**Let's go through each column and review the data within to confirm there aren't other gotchas hiding**.

### Geo-related features

`police_district`, `neighborhoods_sffind_boundaries` and `supervisor_district` are all related to the lat and long

In [ ]:
df[(df.lat == '0') | (df.long == '0')].shape[0]

In [ ]:
df[(df.lat == '0') & df.police_district.isnull() 
   & df.neighborhoods_sffind_boundaries.isnull() 
   & df.supervisor_district.isnull()].shape[0]

**Most records with a (lat, long) of 0 are also missing the associated geographic dimensions.**

For questions unrelated to geography, this doesn't matter, so let's move on.

Next, we check for missing values that appear as empty strings (the checks above would identify only `NA` values):

In [ ]:
df[df.neighborhoods_sffind_boundaries.str.len() == 0].shape[0]

We'll run this check quite a bit on other fields, so let's abstract this as its own function

In [ ]:
def check_for_emtpy_strings(df, field):
    return df[df[field].str.len() == 0].shape[0]

In [ ]:
check_for_emtpy_strings(df, 'neighborhoods_sffind_boundaries')

In [ ]:
df.neighborhoods_sffind_boundaries.value_counts().head(20)

In [ ]:
# Plot the top 10 neighborhoods by 311 calls of any type
# This isn't normalized by population, but gives us a sense of call volume
df.neighborhoods_sffind_boundaries.value_counts().head(10).iloc[::-1].plot.barh()

In [ ]:
check_for_emtpy_strings(df, 'police_district')

### `supervisor_district`

In addition to other data-cleanliness tasks, I'm also interested to see whether all 311 calls have a valid supervisor district attached. We can pull in the list of current supervisor districts from `DataSF`, as well.

In [ ]:
supervisor_district_sfdata_resource_id = 'keex-zmn4'
supervisor_district_data = client.get(supervisor_district_sfdata_resource_id)

In [ ]:
supervisor_district_df = DataFrame.from_records(supervisor_district_data)

In [ ]:
supervisor_districts = set(supervisor_district_df.supervisor.unique())
supervisor_districts

In [ ]:
# The - operator lets us find the difference between two sets
invalid_supervisor_districts = set(df.supervisor_district.unique()) - supervisor_districts

In [ ]:
df[df.supervisor_district.isin(invalid_supervisor_districts)].supervisor_district.value_counts()

We see roughly 1,000 records with a supervisor district of 0. If we dig into supervisor district later, let's take note

In [ ]:
columns_to_review.remove('lat')
columns_to_review.remove('long')
columns_to_review.remove('neighborhoods_sffind_boundaries')
columns_to_review.remove('supervisor_district')
columns_to_review.remove('police_district')

###  `address`

There is only 1 record in this dataset where address is null, and no records where address is an empty string:

In [ ]:
df[df.address.isnull()].shape[0]

In [ ]:
check_for_emtpy_strings(df, 'address')

In [ ]:
df[df.address.str.len() < 20].address.value_counts()

There are 18 records with an address of `Tara Boss`. I'm unsure what this means, but it's a small issue. Let's move on.

In [ ]:
columns_to_review.remove('address')

### `agency_responsible`

In [ ]:
check_for_emtpy_strings(df, 'agency_responsible')

In [ ]:
df[df.agency_responsible.isnull()].shape[0]

In [ ]:
df.agency_responsible.value_counts().head(10).iloc[::-1].plot.barh()

The `agency_responsible` field seems to be relatively well-maintained, and doesn't appear to be missing data. This does not mean we won't find issues with it later, but let's move on.

In [ ]:
columns_to_review.remove('agency_responsible')

In [ ]:
columns_to_review

### `requested_datetime`, `closed_date` and `updated_datetime`

For some analyses, we want to keep tickets that are not closed, so we keep missing data from this field.

First, we want to convert all fields to datetime types.

In [ ]:
df.requested_datetime = pd.to_datetime(df.requested_datetime)
df.closed_date = pd.to_datetime(df.closed_date)
df.updated_datetime = pd.to_datetime(df.updated_datetime)

Let's also get a sense for when tickets are opened and closed

In [ ]:
plt.figure()
x = df.requested_datetime.dt.month.unique()
opened = df.requested_datetime.dt.month.value_counts().sort_index()
closed = df.closed_date.dt.month.value_counts().sort_index()

plt.plot(x, opened, color='blue')
plt.plot(x, closed, color='red')
plt.legend()

What type of tickets are still open? Are most of these just opened in December, or are there others that remain open for some reason?

In [ ]:
open_tickets = df[df.closed_date.isnull()]
open_tickets.shape[0]

In [ ]:
open_tickets.requested_datetime.dt.month.value_counts().sort_index().plot()

There's a lot more we could investigate here, but let's move on

In [ ]:
columns_to_review.remove('requested_datetime')
columns_to_review.remove('closed_date')
columns_to_review.remove('updated_datetime')

At this point, I like to save the filtered / cleaned DataFrame to a CSV that we can use in the rest of our analysis (typically done in a separate notebook).